In [1]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading

In [2]:
# Initialize mediapipe and text-to-speech
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

In [3]:
engine = pyttsx3.init()

In [4]:
# Text-to-speech function using threading
def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()


In [5]:
# Helper function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

In [6]:
# Start video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

True

In [7]:
# Shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False
elbow_warning_issued = False

In [8]:

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))


In [ ]:

import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading

# Initialize mediapipe and text-to-speech
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# Text-to-speech function using threading
def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Helper function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Start video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False
elbow_warning_issued = False

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))

# Mediapipe pose instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        # Convert to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process pose
        results = pose.process(image)

        # Convert back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Extract relevant joints
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            # Calculate angles
            angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

            # Display shoulder angle
            cv2.putText(image, str(int(angle)),
                        tuple(np.multiply(right_shoulder, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Elbow straightness check
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    alert_text = "Straighten your elbow"
                    speak(alert_text)
                    elbow_warning_issued = True
                cv2.putText(image, "Elbow bent", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                elbow_warning_issued = False

            # Shoulder abduction rep counter
            if angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")

                    if counter >= target_reps:
                        message = "Great job! You smashed your goal"
                        speak(message)
                        cv2.putText(image, message, (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1000)
                        break

            elif angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True

        except Exception as e:
            pass

        # Draw UI
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1746926106.995529  735576 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1746926107.147157  736479 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746926107.170814  736476 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746926107.196476  736480 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


: 

In [1]:
import tensorflow as tf

# Simple TensorFlow operation
a = tf.constant(2)
b = tf.constant(3)
c = a + b
print(c.numpy())

5


In [1]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading
from queue import Queue

# Initialize mediapipe and text-to-speech
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# Queue for text-to-speech messages
speech_queue = Queue()

# Background thread for speaking
def speech_loop():
    while True:
        text = speech_queue.get()
        if text is None:
            break
        engine.say(text)
        engine.runAndWait()
        speech_queue.task_done()

speech_thread = threading.Thread(target=speech_loop, daemon=True)
speech_thread.start()

# Function to enqueue speech
def speak(text):
    speech_queue.put(text)

# Helper function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle
    return angle

# Start video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False
elbow_warning_issued = False

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))

# Mediapipe pose instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        # Convert to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process pose
        results = pose.process(image)

        # Convert back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Extract relevant joints
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            # Calculate angles
            angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

            # Display shoulder angle
            cv2.putText(image, str(int(angle)),
                        tuple(np.multiply(right_shoulder, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Elbow straightness check
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    speak("Straighten your elbow")
                    elbow_warning_issued = True
                cv2.putText(image, "Elbow bent", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                elbow_warning_issued = False

            # Shoulder abduction rep counter
            if angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")

                    if counter >= target_reps:
                        message = "Great job! You smashed your goal"
                        speak(message)
                        cv2.putText(image, message, (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(2000)
                        break

            elif angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True

        except Exception as e:
            pass

        # Draw UI
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    

    # Stop the speech thread
    speech_queue.put(None)
    speech_thread.join()


I0000 00:00:1746925786.833628  732157 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1746925786.983947  732450 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746925787.010957  732450 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746925787.235191  732446 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
Exception in thread Thread-4 (speech_loop):
Traceback (most recent call last):
  File "/Users/wasanotb/anaconda3/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/Users/wasanotb/anaconda3/lib/python3.11/site-packages/ipykernel/i

: 

In [1]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading

# Initialize mediapipe and text-to-speech
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# Text-to-speech function using threading
def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Helper function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Start video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False
elbow_warning_issued = False

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))

# Mediapipe pose instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        # Convert to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process pose
        results = pose.process(image)

        # Convert back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Extract relevant joints
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            # Calculate angles
            angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

            # Display shoulder angle
            cv2.putText(image, str(int(angle)),
                        tuple(np.multiply(right_shoulder, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Elbow straightness check
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    alert_text = "Straighten your elbow"
                    # speak(alert_text)
                    elbow_warning_issued = True
                cv2.putText(image, "Elbow bent", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                elbow_warning_issued = False

            # Shoulder abduction rep counter
            if angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")

                    if counter >= target_reps:
                        message = "Great job! You smashed your goal"
                        # speak(message)
                        cv2.putText(image, message, (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1000)
                        break

            elif angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True

        except Exception as e:
            pass

        # Draw UI
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


I0000 00:00:1746926339.321745  738785 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1746926339.466760  739033 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746926339.489428  739033 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746926339.517027  739029 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Rep: 1
Rep: 2


In [2]:
import os
os.environ['GLOG_minloglevel'] = '2'


In [1]:
import cv2
import numpy as np
import mediapipe as mp

# Initialize Mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Calculate angle function
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Rep tracking
counter = 0
stage = None  # "bent" or "straight"

# User input
target_reps = int(input("Enter number of reps: "))

# Start webcam
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        # Process frame
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get left leg coordinates
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            angle = calculate_angle(hip, knee, ankle)

            # Display angle
            cv2.putText(image, f'{int(angle)}',
                        tuple(np.multiply(knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Rep logic
            if angle > 160:
                if stage == "bent":
                    stage = "straight"
                    counter += 1
                    print(f"Rep: {counter}")
            elif angle < 100:
                if stage != "bent":
                    stage = "bent"

            # Display rep count
            cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
            cv2.putText(image, 'REPS', (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

            # Complete message
            if counter >= target_reps:
                cv2.putText(image, 'All reps done!', (200, 200),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)

        except Exception as e:
            print("Error:", e)

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Knee Extension Tracker', image)

        if cv2.waitKey(10) & 0xFF == ord('q') or counter >= target_reps:
            break

    cap.release()
    cv2.destroyAllWindows()


I0000 00:00:1746962905.678747   38336 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1746962905.796648   38662 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746962905.815377   38660 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'


W0000 00:00:1746962906.261848   38661 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Rep: 1
Rep: 2
Rep: 3
Rep: 4


# t

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
# import pyttsx3
# import threading

# Initialize Mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# # Text-to-speech function using threading (موقّف الآن بالكومنت)
# engine = pyttsx3.init()
# def speak(text):
#     def _speak():
#         engine.say(text)
#         engine.runAndWait()
#     threading.Thread(target=_speak).start()

# حساب الزاوية بين ثلاث نقاط
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - \
              np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle
    return angle

# إعداد الكاميرا
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# عداد التكرارات
counter = 0
stage = None  # "up" or "down"
rep_completed = False
started = False  # <-- لضمان عدم العد مباشرة

# عدد التكرارات المطلوبة
target_reps = int(input("Enter number of reps: "))

# تشغيل Mediapipe Pose
with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        # التحويل إلى RGB للمعالجة
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        # الرجوع إلى BGR للعرض
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # نقاط مفصل الرجل اليسرى
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # حساب الزاوية عند الركبة
            angle = calculate_angle(hip, knee, ankle)

            # عرض الزاوية على الشاشة
            cv2.putText(image, str(int(angle)),
                        tuple(np.multiply(knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)

            # منطق العد: البداية عند الاستقامة
            if angle > 150:
                if not started:
                    started = True  # تجاهل أول استقامة
                else:
                    stage = "up"
                    rep_completed = True

            # العد يتم عند العودة للانثناء
            if angle < 120 and stage == "up":
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")

                    if counter >= target_reps:
                        message = "Excellent! Knee extension complete"
                        # speak(message)  # موقّف الآن
                        cv2.putText(image, message, (30, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1500)
                        break

        except:
            pass

        # مستطيل التكرار
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # رسم الجسم
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks,
                                      mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # عرض الكاميرا
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# إغلاق الكاميرا والنوافذ
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1746964283.028443   50292 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M1
W0000 00:00:1746964283.151485   50797 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746964283.169964   50797 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Rep: 1
Rep: 2
Rep: 3
Rep: 4


In [ ]:
import cv2
import numpy as np
import mediapipe as mp

# Initialize Mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# حساب الزاوية بين ثلاث نقاط
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - \
              np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle
    return angle

# إعداد الكاميرا
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# عداد التكرارات
counter = 0
stage = None  # "up" or "down"
rep_completed = False
started = False  # <-- لضمان عدم العد مباشرة

# عدد التكرارات المطلوبة
target_reps = int(input("Enter number of reps: "))

# تشغيل Mediapipe Pose
with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        # التحويل إلى RGB للمعالجة
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        # الرجوع إلى BGR للعرض
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # نقاط مفصل الرجل اليسرى
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # حساب الزاوية عند الركبة
            angle = calculate_angle(hip, knee, ankle)

            # عرض الزاوية على الشاشة
            cv2.putText(image, str(int(angle)),
                        tuple(np.multiply(knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)

            # إذا كانت الزاوية صغيرة جداً فهذا يعني أنك في وضع الجلوس
            if angle < 120 and not started:
                started = True  # بدأنا في الجلوس

            # منطق العد: البداية عند الاستقامة
            if angle > 150 and started:
                stage = "up"
                rep_completed = True

            # العد يتم عند العودة للانثناء
            if angle < 120 and stage == "up":
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")

                    if counter >= target_reps:
                        message = "Excellent! Knee extension complete"
                        cv2.putText(image, message, (30, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1500)
                        break

        except:
            pass

        # مستطيل التكرار
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # رسم الجسم
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks,
                                      mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # عرض الكاميرا
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# إغلاق الكاميرا والنوافذ
cap.release()
cv2.destroyAllWindows()


لسا يحسب اول جلسة


In [ ]:
import cv2
import numpy as np
import mediapipe as mp

# Initialize Mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# حساب الزاوية بين ثلاث نقاط
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - \
              np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle
    return angle

# إعداد الكاميرا
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# عداد التكرارات
counter = 0
stage = None  # "up" or "down"
rep_completed = False
started = False  # <-- لضمان عدم العد مباشرة

# عدد التكرارات المطلوبة
target_reps = int(input("Enter number of reps: "))

# تشغيل Mediapipe Pose
with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        # التحويل إلى RGB للمعالجة
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        # الرجوع إلى BGR للعرض
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # نقاط مفصل الرجل اليسرى
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # حساب الزاوية عند الركبة
            angle = calculate_angle(hip, knee, ankle)

            # عرض الزاوية على الشاشة
            cv2.putText(image, str(int(angle)),
                        tuple(np.multiply(knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)

            # إذا كانت الزاوية صغيرة جداً فهذا يعني أنك في وضع الجلوس
            if angle < 120 and not started:
                started = True  # بدأنا في الجلوس

            # بعد أن نكون في وضع الجلوس، يمكننا بدء العد
            if started:
                # منطق العد: البداية عند الاستقامة
                if angle > 150 and stage != "up":  # يجب أن نكون قد عدنا للتمدد
                    stage = "up"
                    rep_completed = True

                # العد يتم عند العودة للانثناء
                if angle < 120 and stage == "up":
                    if rep_completed:
                        counter += 1
                        rep_completed = False
                        print(f"Rep: {counter}")

                        if counter >= target_reps:
                            message = "Excellent! Knee extension complete"
                            cv2.putText(image, message, (30, 200),
                                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)
                            cv2.imshow('Mediapipe Feed', image)
                            cv2.waitKey(1500)
                            break

        except:
            pass

        # مستطيل التكرار
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # رسم الجسم
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks,
                                      mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # عرض الكاميرا
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# إغلاق الكاميرا والنوافذ
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1746977859.110615  135596 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1746977859.246104  136271 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746977859.267514  136271 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746977860.266852  136267 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Rep: 1


1   HIToolbox                           0x0000000189f145c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x0000000189f14188 _ZN15MenuBarInstance14EnableAutoShowEv + 60
3   HIToolbox                           0x0000000189e818bc _ZN15MenuBarInstance21UpdateAggregateUIModeE21MenuBarAnimationStylehhh + 1184
4   HIToolbox                           0x0000000189f14004 _ZN15MenuBarInstance19SetFullScreenUIModeEjj + 180
5   AppKit                              0x0000000183d03d30 -[NSApplication _setPresentationOptions:instance:flags:] + 956
6   AppKit                              0x0000000183b9993c -[NSApplication _updateFullScreenPresentationOptionsForInstance:] + 404
7   CoreFoundation                      0x00000001807b6560 __CFNOTIFICATIONCENTER_IS_CALLING_OUT_TO_AN_OBSERVER__ + 148
8   CoreFoundation                      0x0000000180854044 ___CFXRegistrationPost_block_invoke + 88
9   CoreFoundation                      0x0000000180853f8c _CFXRe

# t improve

In [1]:
import cv2
import numpy as np
import mediapipe as mp
# import pyttsx3
# import threading

# Initialize Mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# # Text-to-speech function using threading (موقّف الآن بالكومنت)
# engine = pyttsx3.init()
# def speak(text):
#     def _speak():
#         engine.say(text)
#         engine.runAndWait()
#     threading.Thread(target=_speak).start()

# حساب الزاوية بين ثلاث نقاط
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - \
              np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle
    return angle

# إعداد الكاميرا
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# عداد التكرارات
counter = 0
stage = None  # "up" or "down"
rep_completed = False
started = False  # <-- لضمان عدم العد مباشرة

# عدد التكرارات المطلوبة
target_reps = int(input("Enter number of reps: "))

# تشغيل Mediapipe Pose
with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        # التحويل إلى RGB للمعالجة
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        # الرجوع إلى BGR للعرض
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # نقاط مفصل الرجل اليسرى
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # حساب الزاوية عند الركبة
            angle = calculate_angle(hip, knee, ankle)

            # عرض الزاوية على الشاشة
            cv2.putText(image, str(int(angle)),
                        tuple(np.multiply(knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)

            # منطق العد: بداية الجلوس بعد تأكيد الاستقامة
            if angle > 150:
                if not started:
                    started = True  # تجاهل أول استقامة
                else:
                    stage = "up"
                    rep_completed = True

            # العد يتم عند العودة للانثناء بشكل واضح (زاوية أقل من 120 درجة)
            if angle < 120 and stage == "up":
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")

                    if counter >= target_reps:
                        message = "Excellent! Knee extension complete"
                        # speak(message)  # موقّف الآن
                        cv2.putText(image, message, (30, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1500)
                        break

        except:
            pass

        # مستطيل التكرار
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # رسم الجسم
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks,
                                      mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # عرض الكاميرا
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# إغلاق الكاميرا والنوافذ
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1746964464.014027   51993 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1746964464.133570   52256 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746964464.155170   52261 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746964464.181102   52256 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Rep: 1
Rep: 2
Rep: 3
Rep: 4


# renad

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
# import pyttsx3    # تم تعطيله
# import threading  # تم تعطيله

# === تهيئة Mediapipe ===
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# === دالة حساب الزاوية ===
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# === (تم تعطيل الصوت) ===
# engine = pyttsx3.init()
# def speak(text):
#     def _speak():
#         engine.say(text)
#         engine.runAndWait()
#     threading.Thread(target=_speak).start()

# === بدء التقاط الفيديو ===
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# === متغيرات العد والمنطق ===
counter = 0
stage = "down"       # الحالة الابتدائية: ساق لأسفل
rep_completed = False

# سؤال المستخدم عن عدد التكرارات المستهدفة
target_reps = int(input("Enter number of reps: "))

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)

        # تحويل للصورة لـ RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        # إعادة BGR للرسم
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # استخراج مفاصل الساق اليمنى
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            # حساب زاوية الركبة
            angle = calculate_angle(hip, knee, ankle)

            # عرض الزاوية على الصورة
            cv2.putText(image, str(int(angle)),
                        tuple(np.multiply(knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # طباعة الزاوية والحالة للتصحيح
            print(f"Knee Angle: {angle:.2f}, Stage: {stage}, Reps: {counter}")

            # منطق العد:
            # 1) إذا زاوية > 150 → ساق مرفوعة → ضبط stage = "up"
            if angle > 150:
                stage = "up"
                rep_completed = True

            # 2) إذا زاوية < 120 و stage == "up" → ساق منثنية بعد الرفع → إضافة تكرار
            if angle < 120 and stage == "up" and rep_completed:
                counter += 1
                rep_completed = False
                stage = "down"
                print(f"Rep: {counter}")
                # speak(f"{counter}")  # صوت التكرار (معطل)

                # عند انتهاء الهدف
                if counter >= target_reps:
                    message = "Excellent! Knee extension complete"
                    # speak(message)  # صوت الإكتمال (معطل)
                    cv2.putText(image, message, (50, 200),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)
                    cv2.imshow('Mediapipe Feed', image)
                    cv2.waitKey(2000)
                    break

        # رسم واجهة العد
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # رسم النقاط والاتصالات
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)

        # إنهاء الضغط على 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# تحرير الموارد
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1746964060.015591   46097 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M1
W0000 00:00:1746964060.137165   48146 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746964060.155192   48146 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Knee Angle: 178.23, Stage: down, Reps: 0
Knee Angle: 178.72, Stage: up, Reps: 0
Knee Angle: 178.71, Stage: up, Reps: 0
Knee Angle: 178.87, Stage: up, Reps: 0
Knee Angle: 178.98, Stage: up, Reps: 0
Knee Angle: 178.78, Stage: up, Reps: 0
Knee Angle: 178.90, Stage: up, Reps: 0
Knee Angle: 178.99, Stage: up, Reps: 0
Knee Angle: 179.26, Stage: up, Reps: 0
Knee Angle: 179.26, Stage: up, Reps: 0
Knee Angle: 179.40, Stage: up, Reps: 0
Knee Angle: 179.53, Stage: up, Reps: 0
Knee Angle: 179.52, Stage: up, Reps: 0
Knee Angle: 179.60, Stage: up, Reps: 0
Knee Angle: 179.54, Stage: up, Reps: 0
Knee Angle: 179.65, Stage: up, Reps: 0
Knee Angle: 179.63, Stage: up, Reps: 0
Knee Angle: 179.47, Stage: up, Reps: 0
Knee Angle: 179.41, Stage: up, Reps: 0
Knee Angle: 179.43, Stage: up, Reps: 0
Knee Angle: 179.39, Stage: up, Reps: 0
Knee Angle: 179.42, Stage: up, Reps: 0
Knee Angle: 179.40, Stage: up, Reps: 0
Knee Angle: 179.46, Stage: up, Reps: 0
Knee Angle: 179.57, Stage: up, Reps: 0
Knee Angle: 179.62, Sta

# ideal

In [1]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading

# Initialize Mediapipe and text-to-speech
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# Text-to-speech function using threading
def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Helper function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Start video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Knee extension counter variables
counter = 0
stage = "bent"
rep_completed = False
started = False  # To track if the first rep has started
movement_stage = 0  # To track different stages of movement (0: standing, 1: sitting, 2: extended, 3: seated)

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))

# Mediapipe pose instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        # Convert to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process pose
        results = pose.process(image)

        # Convert back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Extract left leg joints
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # Calculate knee angle
            angle = calculate_angle(left_hip, left_knee, left_ankle)

            # Display angle on screen
            cv2.putText(image, str(int(angle)),
                        tuple(np.multiply(left_knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Logic to detect movement stages
            if angle > 160 and movement_stage == 0:  # Standing position
                movement_stage = 1  # Transition to sitting
            elif angle < 100 and movement_stage == 1:  # Sitting position
                movement_stage = 2  # Transition to extension
            elif angle > 160 and movement_stage == 2:  # Extended position (leg extended)
                movement_stage = 3  # Transition to seated again
            elif angle < 100 and movement_stage == 3:  # Seated again (back to small angle)
                if started:  # If the first rep has already started
                    counter += 1
                    print(f"Rep: {counter}")
                    if counter >= target_reps:
                        message = "Excellent! Knee extension complete"
                        # speak(message)
                        cv2.putText(image, message, (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1000)
                        break
                else:  # If it's the first rep, start counting
                    started = True
                    counter += 1
                    print(f"Rep: {counter}")
                    movement_stage = 0  # Reset the movement stage to start again after rep

        except Exception as e:
            pass

        # Draw UI
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


I0000 00:00:1746965851.405945   62433 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1746965851.526060   63574 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746965851.546512   63574 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746965851.978359   63579 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Rep: 1
Rep: 2
Rep: 3
Rep: 4


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading

# Initialize Mediapipe and text-to-speech
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# Text-to-speech function using threading
def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Helper function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Start video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Knee extension counter variables
counter = 0
stage = "bent"
rep_completed = False
sequence_stage = 0  # Variable to track the sequence of movements

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))

# Mediapipe pose instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        # Convert to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process pose
        results = pose.process(image)

        # Convert back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Extract left leg joints
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # Calculate knee angle
            angle = calculate_angle(left_hip, left_knee, left_ankle)

            # Display angle on screen
            cv2.putText(image, str(int(angle)),
                        tuple(np.multiply(left_knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Rep counter logic with sequence check
            if sequence_stage == 0 and angle > 160:  # Standing position (large angle)
                sequence_stage = 1
            elif sequence_stage == 1 and angle < 100:  # Sitting position (small angle)
                sequence_stage = 2
            elif sequence_stage == 2 and angle > 160:  # Leg extension (large angle)
                sequence_stage = 3
            elif sequence_stage == 3 and angle < 100:  # Returning to sitting position (small angle)
                sequence_stage = 0  # Reset to begin a new rep
                
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")

                    if counter >= target_reps:
                        message = "Excellent! Knee extension complete"
                        speak(message)
                        cv2.putText(image, message, (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1000)
                        break

            # Update the rep_completed flag when the sequence is completed
            if angle > 160 and sequence_stage == 0:
                rep_completed = True

        except Exception as e:
            pass

        # Draw UI
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


I0000 00:00:1746966208.211162   66319 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1746966208.332423   66625 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746966208.353126   66630 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746966208.378445   66626 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


# 🦵 Knee Extension Tracker

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading

# Initialize Mediapipe and text-to-speech
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# Text-to-speech function using threading
def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Helper function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Start video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Knee extension counter variables
counter = 0
stage = "bent"
rep_completed = False

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))

# Mediapipe pose instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        # Convert to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process pose
        results = pose.process(image)

        # Convert back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Extract left leg joints
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # Calculate knee angle
            angle = calculate_angle(left_hip, left_knee, left_ankle)

            # Display angle on screen
            cv2.putText(image, str(int(angle)),
                        tuple(np.multiply(left_knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Rep counter logic
            if angle < 100:
                stage = "bent"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")

                    if counter >= target_reps:
                        message = "Excellent! Knee extension complete"
                        # speak(message)
                        cv2.putText(image, message, (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1000)
                        break

            elif angle > 160 and stage == "bent":
                stage = "straight"
                rep_completed = True

        except Exception as e:
            pass

        # Draw UI
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


I0000 00:00:1746962239.201680   31631 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1746962239.323791   31925 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746962239.343516   31928 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746962239.493904   31927 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Rep: 1
Rep: 2
Rep: 3
Rep: 4


# 2

In [5]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading

# Initialize mediapipe and text-to-speech
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# Text-to-speech function using threading
def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Helper function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Start video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Knee extension counter variables
counter = 0
stage = "down"
rep_completed = False
elbow_warning_issued = False

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))

# Mediapipe pose instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        # Convert to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process pose
        results = pose.process(image)

        # Convert back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Extract relevant joints
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            # Calculate angles
            angle = calculate_angle(right_hip, right_knee, right_ankle)

            # Display knee angle
            cv2.putText(image, str(int(angle)),
                        tuple(np.multiply(right_knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Check for movement of hip and knee to avoid false counting
            hip_y = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
            knee_y = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y

            # Skip rep if the hip or knee has moved significantly (indicating you moved too much)
            if hip_y < 0.4 or knee_y < 0.5:
                continue  # Skip this frame, no rep counted

            # Knee extension rep counter
            if angle > 160:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")

                    if counter >= target_reps:
                        message = "Great job! You smashed your goal"
                        # speak(message)
                        cv2.putText(image, message, (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1000)
                        break

            elif angle < 30 and stage == "down":
                stage = "up"
                rep_completed = True

        except Exception as e:
            pass

        # Draw UI
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


I0000 00:00:1746929773.175900  753716 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M1
W0000 00:00:1746929773.304123  769453 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746929773.326713  769454 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


KeyboardInterrupt: 

# 3

In [1]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading

# Initialize Mediapipe and text-to-speech
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# Text-to-speech function using threading
def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Helper function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Start video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Knee extension counter variables
counter = 0
stage = "bent"
rep_completed = False

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))

# Mediapipe pose instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        # Convert to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process pose
        results = pose.process(image)

        # Convert back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Extract left leg joints
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # Calculate knee angle
            angle = calculate_angle(left_hip, left_knee, left_ankle)

            # Display angle on screen
            cv2.putText(image, str(int(angle)),
                        tuple(np.multiply(left_knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Check for movement of hip and knee to avoid false counting
            hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            knee_y = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y

            # Skip rep if the hip or knee has moved significantly (indicating you moved too much)
            if hip_y < 0.4 or knee_y < 0.5:
                continue  # Skip this frame, no rep counted

            # Rep counter logic
            if angle < 100:
                stage = "bent"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")

                    if counter >= target_reps:
                        message = "Excellent! Knee extension complete"
                        speak(message)
                        cv2.putText(image, message, (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1000)
                        break

            elif angle > 160 and stage == "bent":
                stage = "straight"
                rep_completed = True

        except Exception as e:
            pass

        # Draw UI
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


I0000 00:00:1746930000.263862  770981 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1746930000.421479  771467 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746930000.443570  771466 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746930000.473575  771463 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Rep: 1
Rep: 2
Rep: 3
Rep: 4
